<a href="https://colab.research.google.com/github/AdryGP/ABP-FindMo/blob/main/FindMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FindMo: Manual de Uso**

---



Primero, cargamos los datasets originales, almacenados en un drive público:

In [37]:
# !pip install gdown

!mkdir -p original_datasets/videogames 

!mkdir -p original_datasets/movies 

!mkdir -p system_datasets

import gdown as gd

# Videojuegos
gd.download('https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey', 'original_datasets/videogames/steam_list.csv', quiet=False)

gd.download('https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe', 'original_datasets/videogames/steam_descriptions.csv', quiet=False)

gd.download('https://drive.google.com/uc?id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM', 'original_datasets/videogames/steam_reviews.csv', quiet=False)

# Películas
gd.download('https://drive.google.com/uc?id=1U7OJMy5wmseeeRLiPhwnXRsHzB0Az3-I', 'original_datasets/movies/imbd_movies.csv', quiet=False)

gd.download('https://drive.google.com/uc?id=151yjtQnjfC1PxUiyw1xVXhkmHeBVwL0U', 'original_datasets/movies/metacritic_movies.csv', quiet=False)

gd.download('https://drive.google.com/uc?id=157m8NSM5D7xE-VZ8iqVzNNvQa6rMPf3d', 'original_datasets/movies/metacritic_reviews.csv', quiet=False)

#!wget -O original_datasets/videogames/steam_list.csv 'https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey&export=download'

#!wget -O original_datasets/videogames/steam_descriptions.csv 'https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe&export=download'

#!wget -O original_datasets/videogames/steam_reviews.csv 'https://drive.google.com/uc?export=download&confirm=ApZ6&id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM'

Downloading...
From: https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey
To: /content/original_datasets/videogames/steam_list.csv
5.82MB [00:00, 197MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe
To: /content/original_datasets/videogames/steam_descriptions.csv
94.7MB [00:00, 215MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM
To: /content/original_datasets/videogames/steam_reviews.csv
121MB [00:00, 210MB/s]
Downloading...
From: https://drive.google.com/uc?id=1U7OJMy5wmseeeRLiPhwnXRsHzB0Az3-I
To: /content/original_datasets/movies/imbd_movies.csv
47.9MB [00:00, 91.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=151yjtQnjfC1PxUiyw1xVXhkmHeBVwL0U
To: /content/original_datasets/movies/metacritic_movies.csv
100%|██████████| 636k/636k [00:00<00:00, 66.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=157m8NSM5D7xE-VZ8iqVzNNvQa6rMPf3d
To: /content/original_datasets/movies/met

'original_datasets/movies/metacritic_reviews.csv'

Abrimos los ficheros con Pandas:

In [92]:
import pandas as pd

# Videojuegos
originalSteamList = pd.read_csv('original_datasets/videogames/steam_list.csv')
originalSteamDescriptions = pd.read_csv('original_datasets/videogames/steam_descriptions.csv')
originalSteamReviews = pd.read_csv('original_datasets/videogames/steam_reviews.csv')

# Películas
originalIMBDMoviesWithDescription = pd.read_csv('original_datasets/movies/imbd_movies.csv')
originalMetacriticMovies = pd.read_csv('original_datasets/movies/metacritic_movies.csv')
originalMetacriticReviewa = pd.read_csv('original_datasets/movies/metacritic_reviews.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reducimos el tamaño del dataset, solo considerando aquellos juegos que han vendido más de 20k unidades

In [94]:
originalSteamList = originalSteamList[originalSteamList['owners'] != '0-20000']

Reducimos el tamaño del dataset de cine, solo considerando películas con más de 70 reseñas por usuarios

In [96]:
originalIMBDMoviesWithDescription = originalIMBDMoviesWithDescription[originalIMBDMoviesWithDescription['reviews_from_users'] > 70]

Ahora, unimos datasets y seleccionamos las columnas necesarias:

In [97]:
# Estas líneas crean una copia del csv con un id para cada elemento con el formato que hemos especificado:
# Videojuegos
listadoVideojuegos = originalSteamList.copy()
listadoVideojuegos.insert(0, 'id_obra', range(0, len(listadoVideojuegos)))
listadoVideojuegos = listadoVideojuegos.assign(id_obra=lambda x: 'V'+x.id_obra.astype(str))

# Películas
listadoCineDescripcion = originalIMBDMoviesWithDescription.copy()
listadoCineDescripcion.insert(0, 'id_obra', range(0, len(listadoCineDescripcion)))
listadoCineDescripcion = listadoCineDescripcion.assign(id_obra=lambda x: 'C'+x.id_obra.astype(str))


# Creamos el listado que contiene títulos y descripciones (y guardamos el csv):
# Videojuegos
listadoDescripcionVideojuegos = pd.merge(listadoVideojuegos, originalSteamDescriptions, left_on='appid', right_on='steam_appid')

listadoDescripcionVideojuegos = listadoDescripcionVideojuegos.rename(columns={'name':'nombre', 'short_description':'descripcion'})

listadoDescripcionVideojuegos[['id_obra', 'nombre', 'descripcion']].to_csv('system_datasets/obras_descripcion.csv', index=False)

listadoDescripcionVideojuegos[['id_obra', 'nombre', 'descripcion']]

,id_obra,nombre,descripcion
0,V0,Counter-Strike,Play the world's number 1 online action game. ...
1,V1,Team Fortress Classic,One of the most popular online action games of...
2,V2,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...
3,V3,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...
4,V4,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...
...,...,...,...
8474,V8474,BLOCKADE Classic,BLOCKADE Classic! First Person Shooter in an e...
8475,V8475,Arena Stars,"Choose your Hero, build your Team and take on ..."
8476,V8476,Burning Daylight,"Waking up naked in a slaughterhouse, you must ..."
8477,V8477,KillStreak.tv,KILLSTREAK.tv is a quick fire 3rd-person multi...


In [98]:
# Cine:
listadoCineDescripcion = listadoCineDescripcion.rename(columns={'title':'nombre', 'description':'descripcion'})
listadoCineDescripcion[['id_obra', 'nombre', 'descripcion']].dropna().to_csv('system_datasets/obras_descripcion.csv', index=False, mode='a', header=False)
listadoCineDescripcion[['id_obra', 'nombre', 'descripcion']].dropna()


,id_obra,nombre,descripcion
47,C0,Nascita di una nazione,The Stoneman family finds its friendship with ...
76,C1,Intolerance,"The story of a poor young woman, separated by ..."
147,C2,Giglio infranto,"A frail waif, abused by her brutal boxer fathe..."
165,C3,Il gabinetto del dottor Caligari,"Hypnotist Dr. Caligari uses a somnambulist, Ce..."
190,C4,Dr. Jekyll e Mr. Hyde,Dr. Henry Jekyll experiments with scientific m...
...,...,...,...
85811,C9130,Falling Inn Love - Ristrutturazione con amore,When city girl Gabriela spontaneously enters a...
85817,C9131,Paranormal Investigation,When a young man becomes possessed after playi...
85837,C9132,VFW,A group of old war veterans put their lives on...
85839,C9133,Coffee & Kareem,Twelve-year-old Kareem Manning hires a crimina...


In [11]:
# Creamos el listado que contiene los comentarios asociados a cada obra:
listadoComentariosVideojuegos = pd.merge(listadoDescripcionVideojuegos, originalSteamReviews, left_on='nombre', right_on='title')

listadoComentariosVideojuegos = listadoComentariosVideojuegos.rename(columns={'review':'comentario', 'recommendation':'valoracion'})

listadoComentariosVideojuegos[['id_obra', 'comentario', 'valoracion']].to_csv('system_datasets/obras_comentarios.csv', index=False)

listadoComentariosVideojuegos[['id_obra', 'comentario', 'valoracion']]

,id_obra,comentario,valoracion
0,V21,I used to play this game a lot back when it ca...,Recommended
1,V21,Product received for free. 'No hope. No cure. ...,Recommended
2,V21,cool,Recommended
3,V21,"Well ya see ""TAAAAAAAAAAAAAANK""",Recommended
4,V21,Nice game it's a pity that Gaben doesn't creat...,Recommended
...,...,...,...
434874,V8300,Well having played for a bit now I am pleased ...,Recommended
434875,V8300,runs well without any problem / bug / crash / ...,Recommended
434876,V8300,NO DRM FREE DLC JIGGLE TIDDIES,Recommended
434877,V8300,Pros 2 New Weapon Types introduced with newer ...,Recommended


Preprocesado:

In [99]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

originalData = pd.read_csv('system_datasets/obras_descripcion.csv')

for row in originalData.itertuples():
    
    
    text = word_tokenize(row[3]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalpha()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = originalData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id_obra,nombre,descripcion,processed_text
0,V0,Counter-Strike,Play the world's number 1 online action game. ...,play world number onlin action game engag incr...
1,V1,Team Fortress Classic,One of the most popular online action games of...,one popular onlin action game time team fortre...
2,V2,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,enlist intens brand axi alli teamplay set wwii...
3,V3,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,enjoy multiplay game deathmatch classic dmc va...
4,V4,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,return black mesa research facil one militari ...
...,...,...,...,...
17592,C9130,Falling Inn Love - Ristrutturazione con amore,When city girl Gabriela spontaneously enters a...,when citi girl gabriela spontan enter contest ...
17593,C9131,Paranormal Investigation,When a young man becomes possessed after playi...,when young man becom possess play ouija game g...
17594,C9132,VFW,A group of old war veterans put their lives on...,A group old war veteran put live line defend y...
17595,C9133,Coffee & Kareem,Twelve-year-old Kareem Manning hires a crimina...,kareem man hire crimin scare mom new boyfriend...


In [100]:
preprocessedData.iloc[0]['processed_text']

'play world number onlin action game engag incred realist brand terrorist warfar wildli popular game alli teammat complet strateg mission take enemi site rescu hostag your role affect team success your team success affect role'

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


In [102]:
textsBoW.shape

(17597, 19569)

In [ ]:
print(textsBoW)

In [ ]:
bagOfWordsModel.get_feature_names()

In [105]:
from sklearn.metrics import pairwise_distances

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')

In [106]:
print(distance_matrix.shape)
print(type(distance_matrix))

(17597, 17597)
<class 'numpy.ndarray'>


In [147]:
searchTitle = "Portal" # Obra base para las recomendaciones (Otro: "The Elder Scrolls V: Skyrim")
indexOfTitle = preprocessedData[preprocessedData['nombre']==searchTitle].index.values[0]
indexOfTitle

17

In [149]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))

In [150]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])

In [151]:
top_scores = ordered_scores[1:11]
top_scores

[(7586, 0.6823408451925325),
 (12531, 0.746000135424382),
 (6866, 0.765500156746941),
 (5292, 0.7855029059030173),
 (11625, 0.802166963326994),
 (3211, 0.8102726372868321),
 (8054, 0.8155574752816116),
 (10753, 0.8165194937914994),
 (11167, 0.8166121189348645),
 (3605, 0.820952524053468)]

In [152]:
top_indexes = [i[0] for i in top_scores]
top_indexes

[7586, 12531, 6866, 5292, 11625, 3211, 8054, 10753, 11167, 3605]

In [154]:
preprocessedData[['id_obra','nombre', 'descripcion']].iloc[top_indexes]

,id_obra,nombre,descripcion
7586,V7586,Bridge Constructor Portal,Enter the Aperture Science Enrichment Center a...
12531,C4060,Versus,There are 666 portals that connect this world ...
6866,V6866,Artifact,A collaboration between legendary game designe...
5292,V5292,The Lab,"Welcome to The Lab, a compilation of Valve’s r..."
11625,C3153,Essere John Malkovich,A puppeteer discovers a portal that leads lite...
3211,V3211,Horizon Shift,Horizon Shift is an explosive wave based singl...
8054,V8054,Supraland,"Try the demo! A mix between Portal, Zelda and ..."
10753,C2279,La storia infinita 2,A young boy with a distant father enters a wor...
11167,C2694,Nightmare - Nuovo incubo,A demonic force has chosen Freddy Krueger as i...
3605,V3605,Inbetween Land,Open the portal to a strange world and save yo...


Guardar la matriz de distancias (para no tener que procesarla cada vez) [Pesa mucho, parece que no se puede descargar]

In [35]:
from numpy import savetxt
savetxt('distance_matrix_only_games.csv', distance_matrix, delimiter=',')